In [29]:
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from tensorflow import keras
from keras import Sequential
from keras.layers import Dense,Dropout
import matplotlib.pyplot as plt
import keras_tuner as kt
from tensorflow.keras.callbacks import EarlyStopping

In [2]:
df=pd.read_csv('https://raw.githubusercontent.com/Unstopable18/Datasets/main/diabetes.csv')
df.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [3]:
df.corr()['Outcome']

Pregnancies                 0.221898
Glucose                     0.466581
BloodPressure               0.065068
SkinThickness               0.074752
Insulin                     0.130548
BMI                         0.292695
DiabetesPedigreeFunction    0.173844
Age                         0.238356
Outcome                     1.000000
Name: Outcome, dtype: float64

In [4]:
X=df.iloc[:,:-1].values
y=df.iloc[:,-1].values

In [5]:
scaler=StandardScaler()
X=scaler.fit_transform(X)
X


array([[ 0.63994726,  0.84832379,  0.14964075, ...,  0.20401277,
         0.46849198,  1.4259954 ],
       [-0.84488505, -1.12339636, -0.16054575, ..., -0.68442195,
        -0.36506078, -0.19067191],
       [ 1.23388019,  1.94372388, -0.26394125, ..., -1.10325546,
         0.60439732, -0.10558415],
       ...,
       [ 0.3429808 ,  0.00330087,  0.14964075, ..., -0.73518964,
        -0.68519336, -0.27575966],
       [-0.84488505,  0.1597866 , -0.47073225, ..., -0.24020459,
        -0.37110101,  1.17073215],
       [-0.84488505, -0.8730192 ,  0.04624525, ..., -0.20212881,
        -0.47378505, -0.87137393]])

In [6]:
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=42)


In [7]:
y_train.shape

(614,)

In [ ]:
model=Sequential()
model.add(Dense(32,activation='relu',input_dim=8))
model.add(Dense(1,activation='sigmoid'))
model.compile(optimizer='Adam',loss='binary_crossentropy',metrics=['Accuracy'])
history=model.fit(X,y,epochs=100,batch_size=1,validation_data=(X_test,y_test))

In [ ]:

f=plt.figure()
f.set_figheight(5)
f.set_figwidth(12)

plt.subplot(1, 2, 1)
plt.plot(history.history['Accuracy'],label='Train',linestyle='--')
plt.plot(history.history['val_Accuracy'],label='Test')
plt.xlabel('Accuracy')

plt.subplot(1, 2, 2)
plt.plot(history.history['loss'],label='Train',linestyle='--')
plt.plot(history.history['val_loss'],label='Test')
plt.xlabel('Loss')

plt.legend()

***

### Optimizer Tunning

In [ ]:
def build_model(hp):
    model=Sequential()
    model.add(Dense(32,activation='relu',input_dim=8))
    model.add(Dense(1,activation='sigmoid'))
    optimizer=hp.Choice('optimizer',values=['sgd','rmsprop','adam','adamw','adadelta','adagrad','adamax','adafactor','nadam','ftrl'])
    model.compile(optimizer=optimizer,loss='binary_crossentropy',metrics=['accuracy'])
    return model

In [ ]:
tuner=kt.RandomSearch(build_model,objective='val_accuracy',max_trials=10,directory="hyperTest",project_name="modelHyperTest",)

In [ ]:
tuner.search(X_train,y_train,epochs=100,batch_size=1,validation_data=(X_test,y_test))

In [ ]:
best_hps = tuner.get_best_hyperparameters()[0]
print(best_hps.values)

best_model = tuner.get_best_models(num_models=1)[0]
best_model.summary()

***

In [12]:
def build_model(hp):
    model=Sequential()
    units=hp.Int('units',min_value=8,max_value=128,step=1)
    model.add(Dense(units,activation='relu',input_dim=8))
    model.add(Dense(1,activation='sigmoid'))
    optimizer=hp.Choice('optimizer',values=['sgd','rmsprop','adam','adamw','adadelta','adagrad','adamax','nadam','ftrl'])
    model.compile(optimizer=optimizer,loss='binary_crossentropy',metrics=['accuracy'])
    return model

In [14]:
tuner=kt.RandomSearch(build_model,objective='val_accuracy',max_trials=10,directory="hyperTest",project_name="nodesHyperTest",)

In [15]:
tuner.search(X_train,y_train,epochs=10,validation_data=(X_test,y_test))

Trial 10 Complete [00h 00m 04s]
val_accuracy: 0.7857142686843872

Best val_accuracy So Far: 0.7857142686843872
Total elapsed time: 00h 00m 37s
INFO:tensorflow:Oracle triggered exit


In [17]:
best_hps = tuner.get_best_hyperparameters()[0]
print(best_hps.values)

best_model = tuner.get_best_models(num_models=1)[0]
best_model.summary()

{'units': 52, 'optimizer': 'nadam'}
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 52)                468       
                                                                 
 dense_1 (Dense)             (None, 1)                 53        
                                                                 
Total params: 521
Trainable params: 521
Non-trainable params: 0
_________________________________________________________________


***

In [18]:
def build_model(hp):
    model=Sequential()

    model.add(Dense(52,activation='relu',input_dim=8))

    for i in range(hp.Int('num_layers',min_value=1,max_value=10)):
        model.add(Dense(52,activation='relu',input_dim=8))

    model.add(Dense(1,activation='sigmoid'))
    model.compile(optimizer='nadam',loss='binary_crossentropy',metrics=['accuracy'])
    return model

In [20]:
tuner=kt.RandomSearch(build_model,objective='val_accuracy',max_trials=10,directory="hyperTest",project_name="layerHyperTest",)

In [21]:
tuner.search(X_train,y_train,epochs=10,validation_data=(X_test,y_test))

Trial 8 Complete [00h 00m 05s]
val_accuracy: 0.7792207598686218

Best val_accuracy So Far: 0.7922077775001526
Total elapsed time: 00h 00m 54s
INFO:tensorflow:Oracle triggered exit


In [22]:
best_hps = tuner.get_best_hyperparameters()[0]
print(best_hps.values)

best_model = tuner.get_best_models(num_models=1)[0]
best_model.summary()

{'num_layers': 7}
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 52)                468       
                                                                 
 dense_1 (Dense)             (None, 52)                2756      
                                                                 
 dense_2 (Dense)             (None, 52)                2756      
                                                                 
 dense_3 (Dense)             (None, 52)                2756      
                                                                 
 dense_4 (Dense)             (None, 52)                2756      
                                                                 
 dense_5 (Dense)             (None, 52)                2756      
                                                                 
 dense_6 (Dense)             (None, 52

***

In [30]:
def build_model(hp):
    model=Sequential()
    counter=0
    for i in range(hp.Int('num_layers',min_value=1,max_value=10)):
        if counter==0:
            model.add(Dense(hp.Int('units'+str(i),min_value=8,max_value=128,step=8),activation=hp.Choice('activation',values=['tanh','sigmoid','relu']),input_dim=8))
            model.add(Dropout(hp.Choice('dropout'+str(i),values=[0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9])))
        else:
            model.add(Dense(hp.Int('units'+str(i),min_value=8,max_value=128,step=8),activation=hp.Choice('activation',values=['tanh','sigmoid','relu'])))
            model.add(Dropout(hp.Choice('dropout'+str(i),values=[0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9])))
        counter+=1
    model.add(Dense(1,activation='sigmoid'))
    model.compile(optimizer=hp.Choice('optimizer',values=['sgd','rmsprop','adam','adamw','adadelta','adagrad','adamax','nadam','ftrl']),loss='binary_crossentropy',metrics=['accuracy'])
    return model

In [31]:
tuner=kt.RandomSearch(build_model,objective='val_accuracy',max_trials=10,directory="hyperTest",project_name="finalDropHyperTest",)

In [32]:
tuner.search(X_train,y_train,epochs=10,validation_data=(X_test,y_test))

Trial 10 Complete [00h 00m 03s]
val_accuracy: 0.6428571343421936

Best val_accuracy So Far: 0.7467532753944397
Total elapsed time: 00h 00m 40s
INFO:tensorflow:Oracle triggered exit


In [33]:
best_hps = tuner.get_best_hyperparameters()[0]
print(best_hps.values)

best_model = tuner.get_best_models(num_models=1)[0]
best_model.summary()

{'num_layers': 3, 'units0': 56, 'activation': 'tanh', 'dropout0': 0.1, 'optimizer': 'adamax', 'units1': 48, 'dropout1': 0.7, 'units2': 72, 'dropout2': 0.7, 'units3': 24, 'dropout3': 0.8, 'units4': 64, 'dropout4': 0.4, 'units5': 48, 'dropout5': 0.1, 'units6': 24, 'dropout6': 0.3, 'units7': 128, 'dropout7': 0.6, 'units8': 128, 'dropout8': 0.1, 'units9': 40, 'dropout9': 0.8}
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 56)                504       
                                                                 
 dropout (Dropout)           (None, 56)                0         
                                                                 
 dense_1 (Dense)             (None, 48)                2736      
                                                                 
 dropout_1 (Dropout)         (None, 48)                0         
           